Downloaded from https://bkms.brenda-enzymes.org/download.php , release 2023.1 .
Cliked on the disk icon. 

In [1]:
%%capture
%%bash

cd brenda/

wget https://bkms.brenda-enzymes.org/download/Reactions_BKMS.tar.gz

tar -xzvf Reactions_BKMS.tar.gz

rm Reactions_BKMS.tar.gz

In [1]:
import pandas as pnd

table = pnd.read_csv('brenda/Reactions_BKMS.csv', sep='\t', low_memory=False)
table = table.set_index('ID', drop=True, verify_integrity=True)

# useful columns are : ìEC_Number', 'Reaction_ID_BRENDA', 'Reaction_ID_KEGG', 'Reaction_ID_MetaCyc', 'Reaction_ID_SABIO_RK' 
table = table [['EC_Number', 'Reaction_ID_BRENDA', 'Reaction_ID_KEGG', 'Reaction_ID_MetaCyc', 'Reaction_ID_SABIO_RK']]
table.head()

,EC_Number,Reaction_ID_BRENDA,Reaction_ID_KEGG,Reaction_ID_MetaCyc,Reaction_ID_SABIO_RK
ID,,,,,
1,5.3.3.3,BR1,R03031,VINYLACETYL-COA-DELTA-ISOMERASE-RXN,NaN
2,6.3.2.1,BR101,R02473,PANTOATE-BETA-ALANINE-LIG-RXN,2406
3,6.3.2.10,BR102,R04573,6.3.2.10-RXN,4244
4,6.3.2.4,BR102,R04573,6.3.2.10-RXN,4244
5,6.3.2.11,BR103,R01164,CARNOSINE-SYNTHASE-RXN,NaN


## make something_to_brenda (biochemistry)


In [2]:
ec_to_brenda = {}

for index, row in table.iterrows(): 
    if type(row['EC_Number']) == str and type(row['Reaction_ID_BRENDA']) == str:  # exclude nan (empty) cells:
        for j in row['EC_Number'].split(','):
            if j not in ec_to_brenda.keys(): 
                ec_to_brenda[j] = set()
            for i in  row['Reaction_ID_BRENDA'].split(','):
                ec_to_brenda[j].add( i)
                

In [3]:
kegg_to_brenda = {}

for index, row in table.iterrows(): 
    if type(row['Reaction_ID_KEGG']) == str and type(row['Reaction_ID_BRENDA']) == str:  # exclude nan (empty) cells:
        for j in row['Reaction_ID_KEGG'].split(','):
            if j not in kegg_to_brenda.keys(): 
                kegg_to_brenda[j] = set()
            for i in  row['Reaction_ID_BRENDA'].split(','):
                kegg_to_brenda[j].add( i)
                

In [4]:
metacyc_to_brenda = {}

for index, row in table.iterrows(): 
    if type(row['Reaction_ID_MetaCyc']) == str and type(row['Reaction_ID_BRENDA']) == str:  # exclude nan (empty) cells:
        for j in row['Reaction_ID_MetaCyc'].split(','):
            if j not in metacyc_to_brenda.keys(): 
                metacyc_to_brenda[j] = set()
            for i in  row['Reaction_ID_BRENDA'].split(','):
                metacyc_to_brenda[j].add( i)
                

In [5]:
sabiork_to_brenda = {} 

for index, row in table.iterrows(): 
    if type(row['Reaction_ID_SABIO_RK']) == str and type(row['Reaction_ID_BRENDA']) == str:  # exclude nan (empty) cells:  
        for j in row['Reaction_ID_SABIO_RK'].split(','):
            if j not in sabiork_to_brenda.keys(): 
                sabiork_to_brenda[j] = set()
            for i in  row['Reaction_ID_BRENDA'].split(','):
                sabiork_to_brenda[j].add( i)
              

In [6]:
import copy
import pickle


def extend_with_brenda(something_to_others, ec_to_brenda, kegg_to_brenda, metacyc_to_brenda, table):

    something_to_others_basename = something_to_others
    
    # read something_to_others
    with open(f'reactome_dicts/{something_to_others}.pickle', 'rb') as handler:
        something_to_others = pickle.load(handler)


    something_to_others_brenda = copy.deepcopy(something_to_others)
    for something_id in something_to_others.keys():  # something_id = bigg or seed ID.
        something_to_others_brenda[something_id]['brenda.biochemistry'] = set()
        
        # for 'brenda.biochemistry', hook to EC, KEGG, MetaCyc, SabioRK.
        # EC:
        for ec_code in something_to_others[something_id]['ec-code']:
            try:  
                brenda_ids = ec_to_brenda[ec_code]
                for brenda_id in brenda_ids:
                    something_to_others_brenda[something_id]['brenda.biochemistry'].add(brenda_id)
            except:  
                continue
        # KEGG:
        for kegg_code in something_to_others[something_id]['kegg.reaction']:
            try:  
                brenda_ids = kegg_to_brenda[kegg_code]
                for brenda_id in brenda_ids:
                    something_to_others_brenda[something_id]['brenda.biochemistry'].add(brenda_id)
            except:  
                continue
        # MetaCyc:
        for matacyc_code in something_to_others[something_id]['biocyc']:
            try:  
                brenda_ids = metacyc_to_brenda[matacyc_code]
                for brenda_id in brenda_ids:
                    something_to_others_brenda[something_id]['brenda.biochemistry'].add(brenda_id)
            except:  
                continue
        # SabioRK:
        for sabiork_code in something_to_others[something_id]['sabiork.reaction']:
            try:  
                brenda_ids = sabiork_to_brenda[sabiork_code]
                for brenda_id in brenda_ids:
                    something_to_others_brenda[something_id]['brenda.biochemistry'].add(brenda_id)
            except:  
                continue
                

    # create a set of all the ECs appearing also in brenda.
    ec_in_brenda = set()   
    for i in table['EC_Number'].to_list():
        if type(i) == str: 
            for ec_code in i.split(','):
                ec_in_brenda.add(ec_code)
                
    # for 'brenda', hook to EC:
    for something_id in something_to_others.keys():  # something_id = bigg or seed ID.
        something_to_others_brenda[something_id]['brenda'] = set()
        
        for ec_code in something_to_others[something_id]['ec-code']:
            if ec_code in ec_in_brenda:  # check if this already annotated EC is appearing also in brenda.
                something_to_others_brenda[something_id]['brenda'].add(ec_code)
      
                
    # save sopmething to others extended with pubchem
    with open(f'brenda_dicts/{something_to_others_basename}.pickle', 'wb') as handle:
        pickle.dump(something_to_others_brenda, handle)
    
    
    return something_to_others_brenda
    

In [7]:

bigg_to_others =           extend_with_brenda('bigg_to_others', ec_to_brenda, kegg_to_brenda, metacyc_to_brenda, table)
bigg_to_others_extended =  extend_with_brenda('bigg_to_others_extended', ec_to_brenda, kegg_to_brenda, metacyc_to_brenda, table)
seed_to_others =           extend_with_brenda('seed_to_others', ec_to_brenda, kegg_to_brenda, metacyc_to_brenda, table)
seed_to_others_extended =  extend_with_brenda('seed_to_others_extended', ec_to_brenda, kegg_to_brenda, metacyc_to_brenda, table)

## testing area

In [9]:
bigg_to_others['GDH']

{'kegg.reaction': {'R00243'},
 'biocyc': {'GLUTAMATE-DEHYDROGENASE-RXN'},
 'seed.reaction': {'rxn00182', 'rxn19827', 'rxn30075', 'rxn34790', 'rxn34793'},
 'bigg.reaction': {'GDH',
  'GDH1',
  'GDHm',
  'GLUDxi',
  'R_GDH',
  'R_GDH1',
  'R_GDHm',
  'R_GLUDxi'},
 'rhea': {'15133', '15134', '15135', '15136'},
 'sabiork.reaction': {'755'},
 'ec-code': {'1.4.1.14', '1.4.1.2', '1.4.1.3', '1.4.1.4'},
 'metanetx.reaction': {'MNXR144972'},
 'reactome': {'R-HSA-70589.5', 'R-HSA-70600.5'},
 'brenda.biochemistry': {'BR22683',
  'BR33794',
  'BR36862',
  'BR36868',
  'BR36869',
  'BR36871',
  'BR47378',
  'BR48399',
  'BR48399_WOP',
  'BR50099',
  'BS232308',
  'BS258944',
  'BS397591_WOP',
  'BS408366'},
 'brenda': {'1.4.1.14', '1.4.1.2', '1.4.1.3', '1.4.1.4'}}

In [11]:

seed_to_others_extended['rxn19827']

{'kegg.reaction': {'R00243'},
 'biocyc': {'GLUTAMATE-DEHYDROGENASE-RXN'},
 'seed.reaction': {'rxn00182', 'rxn19827', 'rxn30075', 'rxn34790', 'rxn34793'},
 'bigg.reaction': {'GDH',
  'GDH1',
  'GDHm',
  'GLUDxi',
  'R_GDH',
  'R_GDH1',
  'R_GDHm',
  'R_GLUDxi'},
 'rhea': {'15133', '15134', '15135', '15136'},
 'sabiork.reaction': {'755'},
 'ec-code': {'1.4.1.14', '1.4.1.2', '1.4.1.3', '1.4.1.4'},
 'metanetx.reaction': {'MNXR144972'},
 'reactome': {'R-HSA-70589.5', 'R-HSA-70600.5'},
 'brenda.biochemistry': {'BR22683',
  'BR33794',
  'BR36862',
  'BR36868',
  'BR36869',
  'BR36871',
  'BR47378',
  'BR48399',
  'BR48399_WOP',
  'BR50099',
  'BS232308',
  'BS258944',
  'BS397591_WOP',
  'BS408366'},
 'brenda': {'1.4.1.14', '1.4.1.2', '1.4.1.3', '1.4.1.4'}}